# Used Car Price Prediction

importing libraries

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline

## Read Data

In [49]:
data = pd.read_csv('./vw.csv')
data.head()

model  year  price transmission  mileage fuelType  tax   mpg  engineSize
0   T-Roc  2019  25000    Automatic    13904   Diesel  145  49.6         2.0
1   T-Roc  2019  26883    Automatic     4562   Diesel  145  49.6         2.0
2   T-Roc  2019  20000       Manual     7414   Diesel  145  50.4         2.0
3   T-Roc  2019  33492    Automatic     4825   Petrol  145  32.5         2.0
4   T-Roc  2019  22900    Semi-Auto     6500   Petrol  150  39.8         1.5

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15157 entries, 0 to 15156
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         15157 non-null  object 
 1   year          15157 non-null  int64  
 2   price         15157 non-null  int64  
 3   transmission  15157 non-null  object 
 4   mileage       15157 non-null  int64  
 5   fuelType      15157 non-null  object 
 6   tax           15157 non-null  int64  
 7   mpg           15157 non-null  float64
 8   engineSize    15157 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 1.0+ MB


All data is complete, no null values

In [51]:
data.isnull().sum()

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

## EDA

In [52]:
data['age']=datetime.now().year-data['year']
data.head(3)

model  year  price transmission  mileage fuelType  tax   mpg  engineSize  \
0   T-Roc  2019  25000    Automatic    13904   Diesel  145  49.6         2.0   
1   T-Roc  2019  26883    Automatic     4562   Diesel  145  49.6         2.0   
2   T-Roc  2019  20000       Manual     7414   Diesel  145  50.4         2.0   

   age  
0    3  
1    3  
2    3

In [53]:
data.select_dtypes(include=['object']).describe()

model transmission fuelType
count   15157        15157    15157
unique     27            3        4
top      Golf       Manual   Petrol
freq     4863         9417     8553

In [54]:
data.select_dtypes(include=['int','float']).describe()

year         price        mileage           tax           mpg  \
count  15157.000000  15157.000000   15157.000000  15157.000000  15157.000000   
mean    2017.255789  16838.952365   22092.785644    112.744277     53.753355   
std        2.053059   7755.015206   21148.941635     63.482617     13.642182   
min     2000.000000    899.000000       1.000000      0.000000      0.300000   
25%     2016.000000  10990.000000    5962.000000     30.000000     46.300000   
50%     2017.000000  15497.000000   16393.000000    145.000000     53.300000   
75%     2019.000000  20998.000000   31824.000000    145.000000     60.100000   
max     2020.000000  69994.000000  212000.000000    580.000000    188.300000   

         engineSize           age  
count  15157.000000  15157.000000  
mean       1.600693      4.744211  
std        0.461695      2.053059  
min        0.000000      2.000000  
25%        1.200000      3.000000  
50%        1.600000      5.000000  
75%        2.000000      6.000000  
max        3.200000     22.000000

In [55]:
plt.figure(figsize=(16,8))
sns.heatmap(data.corr(), annot=True, cmap='YlGnBu')
plt.show()

From the heatmap, it shows the correlation between each feature. We only focus on the price column, the most correlated is 

## Feature Engineering

### Feature with category - OneHotEncoder

In [56]:
data.drop(columns='year',inplace=True)
data.head()

model  price transmission  mileage fuelType  tax   mpg  engineSize  age
0   T-Roc  25000    Automatic    13904   Diesel  145  49.6         2.0    3
1   T-Roc  26883    Automatic     4562   Diesel  145  49.6         2.0    3
2   T-Roc  20000       Manual     7414   Diesel  145  50.4         2.0    3
3   T-Roc  33492    Automatic     4825   Petrol  145  32.5         2.0    3
4   T-Roc  22900    Semi-Auto     6500   Petrol  150  39.8         1.5    3

In [57]:
from sklearn.preprocessing import OneHotEncoder
cat_columns = ['model', 'transmission', 'fuelType']

In [103]:
oneHotEncoder = OneHotEncoder(drop='first')
cat_features = oneHotEncoder.fit_transform(data[cat_columns]).toarray()
cat_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

### Number features - 

In [59]:
from sklearn.preprocessing import StandardScaler
num_columns = ['mileage', 'tax', 'mpg', 'engineSize', 'age']


In [60]:
standardScaler = StandardScaler()
num_features = standardScaler.fit_transform(data[num_columns])
num_features

array([[-0.38720878,  0.50812   , -0.30445949,  0.86490153, -0.84959483],
       [-0.82894758,  0.50812   , -0.30445949,  0.86490153, -0.84959483],
       [-0.69409005,  0.50812   , -0.24581591,  0.86490153, -0.84959483],
       ...,
       [ 2.2653046 ,  1.37452748, -0.86157349, -0.43470151,  4.02134696],
       [ 2.86601642,  0.58688432, -0.54636425, -0.86790253,  5.48262949],
       [ 2.4544456 ,  0.50812   , -0.54636425, -0.86790253,  4.99553531]])

## X and Y

In [61]:
X = np.hstack([cat_features, num_features])
X

array([[ 0.        ,  0.        ,  0.        , ..., -0.30445949,
         0.86490153, -0.84959483],
       [ 0.        ,  0.        ,  0.        , ..., -0.30445949,
         0.86490153, -0.84959483],
       [ 0.        ,  0.        ,  0.        , ..., -0.24581591,
         0.86490153, -0.84959483],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -0.86157349,
        -0.43470151,  4.02134696],
       [ 0.        ,  0.        ,  0.        , ..., -0.54636425,
        -0.86790253,  5.48262949],
       [ 0.        ,  0.        ,  0.        , ..., -0.54636425,
        -0.86790253,  4.99553531]])

In [64]:
y = data["price"].to_numpy()
y

array([25000, 26883, 20000, ...,  1590,  1250,  2295], dtype=int64)

## Modeling

### Splitting training and testing data

In [69]:
from sklearn.model_selection import train_test_split

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [67]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((10609, 36), (4548, 36), (10609,), (4548,))

In [92]:
df = pd.DataFrame(X_train)
df['price'] = y_train
df

0    1    2    3    4    5    6    7    8    9  ...   27   28   29  \
0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
3      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
4      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  1.0  0.0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
10604  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
10605  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
10606  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
10607  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
10608  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

        30        31        32        33        34        35  price  
0      0.0 -0.678770  0.508120  1.506161 -0.001500  0.124594  12995  
1      1.0 -1.044190  0.586884 -0.429077 -1.301104 -1.336689  21995  
2      1.0 -0.970425  0.508120 -0.656321 -0.218101 -0.849595  19436  
3      1.0 -0.563063  0.508120 -0.751617 -0.218101 -0.849595  21366  
4      0.0 -0.260626  0.429356  8.242842 -0.434702 -0.362501  21995  
...    ...       ...       ...       ...       ...       ...    ...  
10604  0.0  1.319600  0.586884 -0.304459  0.864902  0.124594  21850  
10605  1.0 -0.227337  0.508120 -1.022843  0.864902  0.124594  24989  
10606  1.0 -0.795611 -1.460988  0.663160 -1.301104  0.611688   6999  
10607  1.0 -0.788992  0.508120  0.465237 -1.301104 -0.362501  10995  
10608  0.0 -0.543439  0.508120  0.289307 -0.001500 -0.849595  14950  

[10609 rows x 37 columns]

### Random Forest Regressor

In [68]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

In [72]:
randomForestRegressor = RandomForestRegressor(n_estimators=300,random_state=40,n_jobs=-1)

In [74]:
rfr_model = randomForestRegressor.fit(X_train, y_train)

In [75]:
y_pred = rfr_model.predict(X_test)
y_pred

array([ 7283.76666667, 13303.29      , 16248.7       , ...,
       10409.00065911,  8482.17666667, 23478.91333333])

In [76]:
rfr_model.score(X_train, y_train)

0.9934102344156491

In [93]:
rfr_model.score(X_test, y_test)

0.9495496019092977

### Linear Regression Model

In [94]:
from sklearn.linear_model import LinearRegression

In [95]:
lr_model = LinearRegression()

In [97]:
lr_model.fit(X_train, y_train)

LinearRegression()

In [98]:
lr_model.score(X_train, y_train)

0.8879129294368825

In [99]:
lr_model.score(X_test, y_test)

0.8914152316435985